In [1]:
import gensim
from pyspark.ml.feature import Tokenizer, StopWordsRemover, Word2Vec, Word2VecModel

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np

In [2]:
PATH = './data/'
# spark = SparkSession.builder.master("local").appName("WikiParse").getOrCreate()
conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)

In [3]:
def save_model(w2v_model, file_name):
    vectors = w2v_model.getVectors()
    size = len(vectors.first()['vector'])
    amount = vectors.count()

    vectors_collection = vectors.collect()

    f = open(file_name, 'w')

    f.write('{} {}\n'.format(amount, size))

    for row in vectors_collection:
        vec_str = ' '.join([ str(np.round(x, 5)) for x in row['vector']])
        f.write(row['word'] + ' ' + vec_str + '\n')

    f.close()
    
# save_model(custom_model, PATH + 'custom.embeddings.{}'.format(n))

custom_models = dict()

In [16]:
# n = 11

# custom_model = Word2VecModel.load(PATH + "/w2v/word2vec_{}".format(n))
# print('Loaded Word2VecModel: {}'.format(n))
# save_model(custom_model, PATH + 'custom.embeddings.{}'.format(n))
# print('Saved: {}'.format(n))

# custom_model = gensim.models.KeyedVectors.load_word2vec_format(PATH + 'custom.embeddings.{}'.format(n))
# print('Loaded Gensim: {}'.format(n))
# custom_models[n] = custom_model

Loaded Word2VecModel: 11
Saved: 11


In [10]:
eta_model = gensim.models.KeyedVectors.load_word2vec_format(PATH+\
                                                            'ubercorpus.lowercased.tokenized.word2vec.300d.bz2')

/home/serhii/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
for load in [1,3,5,7,9,11]:
    custom_model = gensim.models.KeyedVectors.load_word2vec_format(PATH + 'custom.embeddings.{}'.format(load))
    print('Loaded Gensim: {}'.format(load))
    custom_models[load] = custom_model

/home/serhii/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Loaded Gensim: 1
Loaded Gensim: 3
Loaded Gensim: 5
Loaded Gensim: 7
Loaded Gensim: 9
Loaded Gensim: 11


In [13]:
def test_model(model, num = 1000):
    
    total_num = 0
    counter = 0
    
    start = 21000
    
    s = 2000

    with open(PATH + 'test_vocabulary.txt') as f:
        lines = f.readlines()
        
        test_arr = lines[s:s+num] + lines[start:]

        for line in test_arr:
            test_words = line[:-1].lower().split('\t')
            if test_words[0][0] == ':':
                continue

            w1, w2, w3, w4 = tuple(test_words)

            if w1 in model and w2 in model and w3 in model and w4 in model:
                
                total_num += 1

                result = model.most_similar(positive=[w1, w4], negative=[w3])

                for r in result:
                    if r[0] == w2:
                        counter += 1
                        break
                    
    return np.round(counter / total_num, 4), counter, total_num, len(test_arr)

acc_result = []

for test in [1,3,5,7,9,11]:
    print(test, test_model(custom_models[test]))
    acc_result.append((test, test_model(custom_models[test])))

1 (0.094, 205, 2180, 3982)
3 (0.1754, 475, 2708, 3982)
5 (0.1702, 461, 2708, 3982)
7 (0.1617, 459, 2839, 3982)
9 (0.1997, 575, 2880, 3982)
11 (0.2292, 660, 2880, 3982)


In [14]:
acc_eta = test_model(eta_model)

In [15]:
acc_eta

(0.6714, 2391, 3561, 3982)